In [1]:
#loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# setting plt to inline
%matplotlib inline

In [2]:
# importing sqlite3
import sqlite3

# conneting to db and creating cursor
conn = sqlite3.Connection('data/movies.db')
cur = conn.cursor()

# creating query functions
def fetcha(q):
    return cur.execute(q).fetchall()
def fetcho(q):
    return cur.execute(q).fetchone()

In [3]:
# getting table names
q = """SELECT name FROM sqlite_master 
WHERE type IN ('table','view') 
AND name NOT LIKE 'sqlite_%'
ORDER BY 1"""
fetcha(q)

[('bom_movie_gross',),
 ('imdb_name_basic',),
 ('imdb_name_basics',),
 ('imdb_title_akas',),
 ('imdb_title_basics',),
 ('imdb_title_crew',),
 ('imdb_title_principals',),
 ('imdb_title_ratings',),
 ('rotten_tomatoes_critic_reviews',),
 ('rotten_tomatoes_movies',),
 ('tmdb_movies',),
 ('tn_movie_budgets',)]

In [5]:
q = "PRAGMA table_info('rotten_tomatoes_movies')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 1),
 (1, 'rotten_tomatoes_link', 'TEXT', 0, None, 0),
 (2, 'movie_title', 'TEXT', 0, None, 0),
 (3, 'movie_info', 'TEXT', 0, None, 0),
 (4, 'critics_consensus', 'TEXT', 0, None, 0),
 (5, 'content_rating', 'TEXT', 0, None, 0),
 (6, 'genres', 'TEXT', 0, None, 0),
 (7, 'directors', 'TEXT', 0, None, 0),
 (8, 'authors', 'TEXT', 0, None, 0),
 (9, 'actors', 'TEXT', 0, None, 0),
 (10, 'original_release_date', 'TEXT', 0, None, 0),
 (11, 'streaming_release_date', 'TEXT', 0, None, 0),
 (12, 'runtime', 'REAL', 0, None, 0),
 (13, 'production_company', 'TEXT', 0, None, 0),
 (14, 'tomatometer_status', 'TEXT', 0, None, 0),
 (15, 'tomatometer_rating', 'REAL', 0, None, 0),
 (16, 'tomatometer_count', 'REAL', 0, None, 0),
 (17, 'audience_status', 'TEXT', 0, None, 0),
 (18, 'audience_rating', 'REAL', 0, None, 0),
 (19, 'audience_count', 'REAL', 0, None, 0),
 (20, 'tomatometer_top_critics_count', 'INTEGER', 0, None, 0),
 (21, 'tomatometer_fresh_critics_count', 'INTEGER', 0, N

In [6]:
q = "PRAGMA table_info('tn_movie_budgets')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 1),
 (1, 'id', 'INTEGER', 0, None, 0),
 (2, 'release_date', 'TEXT', 0, None, 0),
 (3, 'movie', 'TEXT', 0, None, 0),
 (4, 'production_budget', 'TEXT', 0, None, 0),
 (5, 'domestic_gross', 'TEXT', 0, None, 0),
 (6, 'worldwide_gross', 'TEXT', 0, None, 0)]

In [10]:
# creating initial dataframe by joining the four tables and printing first row
q = """SELECT*FROM rotten_tomatoes_movies AS rt
       JOIN tn_movie_budgets AS tn
       ON rt.movie_title = tn.movie
       
       """
df = pd.DataFrame(fetcha(q))
df.columns = [i[0] for i in cur.description]
df.head(1)     

,idx,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,...,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,idx,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,43,73,76,416,17,"Feb 12, 2010",Percy Jackson & the Olympians: The Lightning T...,"$95,000,000","$88,768,303","$223,050,874"


In [11]:
df.loc[df['movie_title'] == 'Mad Max']

,idx,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,...,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,idx,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
2785,9910,m/mad_max,Mad Max,"In a not-too-distant dystopian future, when ma...",Staging the improbable car stunts and crashes ...,R,"Action & Adventure, Cult Movies, Science Ficti...",George Miller,"James McCausland, George Miller","Mel Gibson, Joanne Samuel, Tim Burns, Hugh Kea...",...,7,56,6,5613,14,"Mar 21, 1980",Mad Max,"$200,000","$8,750,000","$99,750,000"


In [28]:
split_list = []
for genre in df.genres:
    split_list.append(genre.split(','))
    
split_list[:5]

[['Action & Adventure', ' Comedy', ' Drama', ' Science Fiction & Fantasy'],
 ['Comedy'],
 ['Action & Adventure', ' Classics', ' Drama'],
 ['Classics', ' Drama', ' Western'],
 ['Art House & International', ' Horror', ' Mystery & Suspense']]

In [36]:
collected_list = []
for _list in split_list:
    for i in range(len(_list)):
        collected_list.append(_list[i])
genres_list = list(set(collected_list))
genres_list

['Science Fiction & Fantasy',
 ' Horror',
 ' Faith & Spirituality',
 ' Art House & International',
 ' Comedy',
 ' Science Fiction & Fantasy',
 'Kids & Family',
 'Animation',
 ' Sports & Fitness',
 ' Special Interest',
 ' Musical & Performing Arts',
 'Romance',
 ' Animation',
 'Drama',
 ' Western',
 ' Cult Movies',
 'Cult Movies',
 'Art House & International',
 ' Mystery & Suspense',
 ' Romance',
 ' Anime & Manga',
 'Documentary',
 'Classics',
 'Western',
 ' Classics',
 'Comedy',
 ' Documentary',
 'Mystery & Suspense',
 'Musical & Performing Arts',
 ' Kids & Family',
 ' Television',
 ' Drama',
 'Horror',
 ' Gay & Lesbian',
 'Action & Adventure']